In [16]:
%load_ext autoreload
%autoreload 2

import os
import oddsapi
import requests
import joblib
import pandas as pd
import numpy as np
from common.utils.odds_utils import get_tournament_winner_odds, get_group_stage_odds, get_group_stage_OU
from common.utils.holdet_utils import get_holdet_data, get_play_off_opps, get_rw
from common.utils.simulation_utils import WorldCupTeam, WorldCupSim
from common.utils.group_stage import get_group_stage, playoffs
from common.utils.dict_utils import eng2dan
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings('ignore')
ROUND = 3
API_KEY = os.getenv("API_KEY")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# load data
df_holdet = get_holdet_data()
df_winner = get_tournament_winner_odds(API_KEY)
df_group_stage = get_group_stage_odds(API_KEY)
df_get_group_stage_OU = get_group_stage_OU(API_KEY)
# df_groups = pd.read_csv("common/ressources/df_groups.csv")[["Team","group"]]
# df_group_stage = df_group_stage[['home_team','away_team','prob1','probx', 'prob2']].merge(df_groups.rename(columns =  {"Team":"home_team"}), on = "home_team")

In [40]:
_df_group_stage = get_group_stage(df_get_group_stage_OU.copy(), df_group_stage.copy(), ROUND, eng2dan)

In [41]:
df_rw = get_rw(df_holdet)
df_rw.rename(columns = {"Player_match": "name1"}, inplace = True)
print(df_holdet.shape)
df_holdet = df_holdet.merge(df_rw, on = "name1", how ="left")
print(df_holdet.shape)
df_holdet = df_holdet[['name1', 'name2', 'country', 'price', 'position', 'Player', 'Team', 'Opp', 'Pos', 'MIN','G', 'A', 'SOG', 'Y','R']]
print(df_holdet.shape)

check names, not all is found
(260, 10)
(262, 39)
(262, 15)


In [31]:
# df_holdet.loc[df_holdet["name1"]=="Jesper Lindstrøm","G"]=0.2
# df_holdet.loc[df_holdet["country"]=="Danmark",:]

In [6]:

df_temp1 = df_holdet[["name1","name2","country"]].merge(_df_group_stage.loc[:,["home_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"home_team":"country"}), on = "country")
df_temp1.rename(columns = {"prob1":f"prob_winning_{round}","probx":f"prob_draw_{round}","prob2":f"prob_lose_{round}","xGH":f"E_goals_{round}","xGA":f"E_goals_against_{round}"}, inplace = True)

df_temp2 = df_holdet[["name1","name2","country"]].merge(_df_group_stage.loc[:,["away_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"away_team":"country"}), on = "country")
df_temp2.rename(columns = {"prob1":f"prob_lose_{round}","probx":f"prob_draw_{round}","prob2":f"prob_winning_{round}","xGA":f"E_goals_{round}","xGH":f"E_goals_against_{round}"}, inplace = True)

df_temp_temp = pd.concat([df_temp1,df_temp2],axis=0)
df_holdet = df_holdet.merge(df_temp_temp, on = ["name1","name2","country"])

In [9]:
# df_winner_old = pd.read_csv("common/ressources/df_winner copy_old")
# df_winner_old = df_winner_old[["name","price","prob_to_win"]]
# df_winner_old["prob_to_win"]=0
# df_winner_old = df_winner_old[~df_winner_old["name"].isin(df_winner["name"])]
# df_winner = pd.concat([df_winner,df_winner_old], axis=0).reset_index(drop=True)

In [42]:
df_winner  = playoffs(df_winner.copy(),_df_group_stage.copy(), eng2dan)
df_holdet = df_holdet.merge(df_winner[['country','R16_prob', 'QF_prob', 'SF_prob', 'Final_prob', 'W_prob', \
                                       'prob_to_win_R16', 'prob_to_win_QF', 'prob_to_win_SF','prob_to_win_Final', \
                                       'xG_R16', 'xG_QF', 'xG_SF', 'xG_Final','xGa_R16', 'xGa_QF', 'xGa_SF', 'xGa_Final']], on = "country")

df_holdet["selected"] = ~df_holdet['G'].isna()

FileNotFoundError: [Errno 2] No such file or directory: 'common/ressources/sims'

In [38]:
df_holdet

,name1,name2,country,price,position,Player,Team,Opp,Pos,MIN,G,A,SOG,Y,R,selected
0,Kylian Mbappe,K. Mbappe,Frankrig,10098000.0,Angreb,Kylian Mbappe,FRA,POL,F,90,0.84,0.34,1.79,0.10,0.00,True
1,Harry Kane,H. Kane,England,9548000.0,Angreb,Harry Kane,ENG,SEN,F,90,0.51,0.17,1.53,0.12,0.00,True
2,Neymar Jr.,N. Jr.,Brasilien,9364000.0,Angreb,Neymar,BRA,KOR,F/M,90,0.72,0.44,1.42,0.18,0.00,True
3,Lionel Messi,L. Messi,Argentina,8695000.0,Angreb,Lionel Messi,ARG,AUS,F/M,90,0.84,0.67,2.13,0.02,0.00,True
4,Memphis Depay,M. Depay,Holland,8327000.0,Angreb,Memphis Depay,NED,USA,F,90,0.43,0.16,1.64,0.09,0.00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,Luka Sucic,L. Sucic,Kroatien,1980000.0,Midtbane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
258,Achraf Dari,A. Dari,Marokko,1980000.0,Forsvar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
259,Ardon Jashari,A. Jashari,Schweiz,1980000.0,Midtbane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
260,Kristijan Jakic,K. Jakic,Kroatien,1980000.0,Midtbane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [11]:
for country in df_holdet["country"].unique():
    xG_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].astype(float).sum()
    xA_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].astype(float).sum()
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].astype(float)/xG_sum
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].astype(float)/xA_sum

    df_holdet.loc[(df_holdet["country"]==country) & (~df_holdet["selected"]),"E_points_total"] = -5000
    df_holdet.loc[(df_holdet["country"]==country) & (~df_holdet["selected"]),"E_points_round1"] = -5000
    for player in df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),"name1"]:
        points = 0
        goal_share = df_holdet.loc[df_holdet["name1"]==player,"G_share"].values[0]
        assist_share = df_holdet.loc[df_holdet["name1"]==player,"A_share"].values[0]
        yellow_prob = df_holdet.loc[df_holdet["name1"]==player,'Y'].astype(float).values[0]
        red_prob = df_holdet.loc[df_holdet["name1"]==player,'R'].astype(float).values[0]
        points += 7000
        points += df_holdet.loc[df_holdet["name1"]==player,f'prob_winning_{i}'].values[0]*25000
        points += df_holdet.loc[df_holdet["name1"]==player,f'prob_draw_{i}'].values[0]*5000
        points += df_holdet.loc[df_holdet["name1"]==player,f'prob_lose_{i}'].values[0]*-15000
        points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*10000
        points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0]*-10000
        points += yellow_prob * -20000
        points += yellow_prob*yellow_prob*-20000
        points += red_prob * -50000
        for idx, game in enumerate(["R16","QF","SF","Final"]):
            prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
            factor = 2/(3*(idx+1))*prob
            points += 7000*factor
            points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xGa_{game}'].values[0])*-10000*factor
            points += prob*df_holdet.loc[df_holdet["name1"]==player,f'prob_to_win_{game}'].values[0]*25000*factor
            points += prob*(1-df_holdet.loc[df_holdet["name1"]==player,f'prob_to_win_{game}'].values[0])*-15000*factor
            points += prob*yellow_prob * -20000*factor
            points += prob*yellow_prob*yellow_prob*-20000*factor
            points += prob*df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0]*10000*factor
            points += prob*red_prob * -50000*factor
                
        if df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Mål":
            points += np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*75000
            for idx, game in enumerate(["R16","QF","SF","Final"]):
                prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                factor = 2/(3*(idx+1))*prob
                points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xGa_{game}'].values[0])*75000*factor
                    
        elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Forsvar":
            points += np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*50000
            points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*175000
            points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000
            for idx, game in enumerate(["R16","QF","SF","Final"]):
                prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                factor = 2/(3*(idx+1))*prob
                points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*50000*factor
                points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*175000*factor
                points += prob*assist_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000*factor
                    
        elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Midtbane":
            points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*150000
            points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000
            for idx, game in enumerate(["R16","QF","SF","Final"]):
                prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                factor = 2/(3*(idx+1))*prob
                points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*150000*factor
                points += prob*assist_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000*factor
                    
        elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Angreb":
            points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*125000
            points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000
            for idx, game in enumerate(["R16","QF","SF","Final"]):
                prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                factor = 2/(3*(idx+1))*prob
                points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*125000*factor
                points += prob*assist_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000*factor

            df_holdet.loc[df_holdet["name1"]==player,"E_points_round1"] = points
                
        df_holdet.loc[df_holdet["name1"]==player,"E_points_total"] = points

In [12]:
df_holdet.sort_values("E_points_total", ascending = False).iloc[0:40]

,name1,name2,country,price,position,Player,Team,Opp,Pos,MIN,...,xG_Final,xGa_R16,xGa_QF,xGa_SF,xGa_Final,selected,G_share,A_share,E_points_total,E_points_round1
108,Cody Gakpo,C. Gakpo,Holland,4869000.0,Angreb,Cody Gakpo,NED,ECU,F/M,90,...,1.135651,0.079265,0.075505,0.163348,0.075505,True,0.285714,0.274510,217796.031400,217796.031400
77,Lionel Messi,L. Messi,Argentina,8431000.0,Angreb,Lionel Messi,ARG,MEX,F/M,90,...,2.759030,0.502538,0.472134,0.447929,0.502538,True,0.320988,0.284314,212062.143224,212062.143224
56,Alisson Becker,A. Becker,Brasilien,5749000.0,Mål,Alisson,BRA,SUI,G,90,...,1.740614,0.243458,0.232596,0.179631,0.179631,True,0.004386,0.017751,206204.713868,NaN
267,Marco Asensio,M. Asensio,Spanien,4744000.0,Midtbane,Marco Asensio,ESP,GER,F/M,90,...,1.354874,0.345145,0.118840,0.212067,0.212067,True,0.305970,0.195876,202315.343945,NaN
104,Denzel Dumfries,D. Dumfries,Holland,5616000.0,Forsvar,Denzel Dumfries,NED,ECU,M/D,90,...,1.135651,0.079265,0.075505,0.163348,0.075505,True,0.111801,0.137255,198180.324718,NaN
0,Kylian Mbappe,K. Mbappe,Frankrig,9659000.0,Angreb,Kylian Mbappe,FRA,DEN,F,90,...,1.564097,0.299134,0.484870,0.466048,0.299134,True,0.350649,0.190476,193343.090956,193343.090956
260,Ferran Torres,F. Torres,Spanien,5834000.0,Angreb,Ferran Torres,ESP,GER,F,90,...,1.354874,0.345145,0.118840,0.212067,0.212067,True,0.358209,0.134021,185827.379605,185827.379605
25,Harry Kane,H. Kane,England,9206000.0,Angreb,Harry Kane,ENG,USA,F,90,...,1.316239,0.412270,0.217911,0.389664,0.217911,True,0.290123,0.147059,185106.580717,185106.580717
54,Vinicius Junior,V. Junior,Brasilien,6674000.0,Angreb,Vinicius Junior,BRA,SUI,F,90,...,1.740614,0.243458,0.232596,0.179631,0.179631,True,0.254386,0.189349,182658.984356,182658.984356
109,Andries Noppert,A. Noppert,Holland,4681000.0,Mål,Andries Noppert,NED,ECU,G,90,...,1.135651,0.079265,0.075505,0.163348,0.075505,True,0.000000,0.000000,181402.951336,NaN


In [15]:
df_holdet.to_csv("common/ressources/df_holdet", index = False)

In [29]:
df_holdet.loc[df_holdet["country"]=="Senegal","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Ghana","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Japan","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Serbien","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Wales","E_points_total"] = 0
df_holdet.loc[df_holdet["country"]=="Senegal","E_points_round1"] = 0
df_holdet.loc[df_holdet["country"]=="Ghana","E_points_round1"] = 0
df_holdet.loc[df_holdet["country"]=="Japan","E_points_round1"] = 0
df_holdet.loc[df_holdet["country"]=="Serbien","E_points_round1"] = 0
df_holdet.loc[df_holdet["country"]=="Wales","E_points_round1"] = 0

In [15]:
df_holdet.sort_values("E_points_total", ascending = False).iloc[0:40]

,name1,name2,country,price,position,Player,Team,Opp,Pos,MIN,...,xG_Final,xGa_R16,xGa_QF,xGa_SF,xGa_Final,selected,G_share,A_share,E_points_total,E_points_round1
0,Kylian Mbappe,K. Mbappe,Frankrig,9282000.0,Angreb,Kylian Mbappe,FRA,AUS,F,90.0,...,1.593750,0.367617,0.594549,0.483353,0.367617,True,0.354167,0.229167,247019.777675,158008.516822
77,Lionel Messi,L. Messi,Argentina,8131000.0,Angreb,Lionel Messi,ARG,KSA,F/M,90.0,...,1.920209,0.575312,0.533931,0.408978,0.408978,True,0.318339,0.232804,242046.078613,160925.202129
207,Robert Lewandowski,R. Lewandowski,Polen,6482000.0,Angreb,Robert Lewandowski,POL,MEX,F,90.0,...,0.090019,0.518865,0.223467,0.065375,0.029840,True,0.625000,0.184466,234414.335680,223745.799464
108,Cody Gakpo,C. Gakpo,Holland,4730000.0,Angreb,Cody Gakpo,NED,SEN,F/M,90.0,...,0.500893,0.133422,0.160107,0.242267,0.160107,True,0.262443,0.286585,234277.793622,135679.584228
51,Neymar Jr.,N. Jr.,Brasilien,9062000.0,Angreb,Neymar,BRA,SRB,F/M,90.0,...,1.766713,0.288051,0.253385,0.217046,0.217046,True,0.282297,0.227586,207527.430854,136237.481856
25,Harry Kane,H. Kane,England,9194000.0,Angreb,Harry Kane,ENG,IRN,F,90.0,...,1.520202,0.519206,0.318721,0.420607,0.318721,True,0.290456,0.149425,204010.560354,132442.922911
104,Denzel Dumfries,D. Dumfries,Holland,5602000.0,Forsvar,Denzel Dumfries,NED,SEN,M/D,90.0,...,0.500893,0.133422,0.160107,0.242267,0.160107,True,0.099548,0.146341,197384.894336,111818.451542
155,Kevin De Bruyne,K. De Bruyne,Belgien,7585000.0,Midtbane,Kevin De Bruyne,BEL,CAN,F/M,90.0,...,0.848258,1.728358,0.226684,0.533374,0.533374,True,0.211618,0.471264,193261.571767,139706.675599
78,Lautaro Martinez,L. Martinez,Argentina,7486000.0,Angreb,Lautaro Martinez,ARG,KSA,F,90.0,...,1.920209,0.575312,0.533931,0.408978,0.408978,True,0.273356,0.084656,190247.555644,126369.544641
267,Marco Asensio,M. Asensio,Spanien,4722000.0,Midtbane,Marco Asensio,ESP,CRC,F/M,90.0,...,1.346297,0.714120,0.287064,0.534727,0.534727,True,0.301818,0.210227,176851.067935,94328.034892
